In [5]:
!pip install dlt[bigquery]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 16.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1


In [6]:
import os
print(os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))



/home/maxkaizo/cdmx_ecobici_usage/terraform/.keys/my-creds.json


In [ ]:
import dlt
import pandas as pd
import hashlib
import gcsfs

# Parámetros
filename = "ecobici_2025-01.csv"
gcs_path = f"dataeng-448500-ecobici-bucket/raw/{filename}"

# Crear cliente GCS (usa GOOGLE_APPLICATION_CREDENTIALS automáticamente si está seteado)
fs = gcsfs.GCSFileSystem(project='dataeng-448500')

# Leer CSV desde GCS
with fs.open(gcs_path, 'rb') as f:
    df = pd.read_csv(f)

# Agregar columnas auxiliares
df["source_file"] = filename
df["ingestion_ts"] = pd.Timestamp.utcnow()

df["ride_id"] = df.apply(
    lambda row: hashlib.md5(
        f"{row['Bici']}_{row['Fecha_Retiro']}_{row['Hora_Retiro']}_{row['Fecha_Arribo']}_{row['Hora_Arribo']}".encode("utf-8")
    ).hexdigest(),
    axis=1
)

# Crear pipeline DLT
pipeline = dlt.pipeline(
    pipeline_name="ecobici_pipeline",
    destination="bigquery",
    dataset_name="ecobici_dataset"
)

# Cargar DataFrame
info = pipeline.run(df, table_name="stg_rides", write_disposition="append")

print(info)


/home/maxkaizo/miniconda3/envs/dataeng/lib/python3.9/site-packages/google/cloud/bigquery/client.py:595: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(
2025-03-26 13:42:35,884|[WARNING]|15410|140687288141312|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'ingestion_ts'. in table 'stg_rides'.
2025-03-26 13:42:38,402|[WARNING]|15410|140687288141312|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'ingestion_ts'. in table 'stg_rides'.


Pipeline ecobici_pipeline load step completed in 1 minute and 58.87 seconds
1 load package(s) were loaded to destination bigquery and into dataset ecobici_dataset
The bigquery destination used None@dataeng-448500 location to store data
Load package 1743018154.2815917 is LOADED and contains no failed jobs
